In [20]:
from gurobipy import GRB
import gurobipy as gb
import pandas as pd
import numpy as np
import autograd as ag

In [21]:
# Create the optimization model
model = gb.Model("Question 1a): TechEssentials Certain Product Line")

In [22]:
# Read costs from CSV files
price_response_df = pd.read_csv(r"C:\Users\gabri\Downloads\price_response.csv")

In [23]:
price_response_df.head(10)

,Product,Intercept,Sensitivity,Capacity
0,Line 1 Product 1,35234.545786,-45.896450,80020.0
1,Line 1 Product 2,37790.240832,-8.227794,89666.0
2,Line 1 Product 3,35675.333217,-7.584436,80638.0
3,Line 2 Product 1,37041.380378,-9.033166,86740.0
4,Line 2 Product 2,36846.140386,-4.427869,84050.0
5,Line 2 Product 3,35827.023747,-2.629060,86565.0
6,Line 3 Product 1,39414.266325,-2.421484,87051.0
7,Line 3 Product 2,35991.951460,-4.000512,85156.0
8,Line 3 Product 3,39313.317031,-2.296622,87588.0


In [24]:
# Extract the "Intercept", "Capacity", and "Sensitivity" column
intercept_values = price_response_df['Intercept'].values.reshape(3, -1)
product_capacity = price_response_df['Capacity'].values.reshape(3, -1)
slope = price_response_df['Sensitivity'].values.reshape(3, -1)

In [25]:
intercept_values

array([[35234.54578551, 37790.24083214, 35675.33321726],
       [37041.38037789, 36846.14038592, 35827.02374655],
       [39414.26632457, 35991.95146022, 39313.31703125]])

In [26]:
product_capacity

array([[80020., 89666., 80638.],
       [86740., 84050., 86565.],
       [87051., 85156., 87588.]])

In [27]:
slope

array([[-45.89644971,  -8.22779417,  -7.58443641],
       [ -9.0331664 ,  -4.42786921,  -2.62906002],
       [ -2.42148392,  -4.0005124 ,  -2.29662237]])

# Part a)

In [28]:
from scipy.optimize import minimize

# Objective function (negated for maximization)
def objective(p):
    p1, p2 = p
    return -1 * (p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2))

# Constraint functions
def constraint1(p):
    return p[1] - p[0]

def constraint2(p):
    return intercept_values[0,0] + slope[0,0] * p[0]

def constraint3(p):
    return intercept_values[0,1] + slope[0,1] * p[1]

# KKT conditions using only constraint1
def kkt_conditions(p, lagrange_multipliers):
    grad_objective = [-(intercept_values[0,0] + 2 * slope[0,0] * p[0]), -(intercept_values[0,1] + 2 * slope[0,1] * p[1])]
    grad_constraint1 = [-1, 1]
    
    # Stationarity condition
    stationarity = [grad_objective[i] + lagrange_multipliers[0] * grad_constraint1[i] for i in range(len(p))]
    
    # Complementary slackness conditions
    complementary_slackness = lagrange_multipliers[0] * constraint1(p)
    
    # Feasibility conditions
    feasibility = constraint1(p)
    
    return stationarity + [complementary_slackness] + [feasibility]

# Initial guess for Lagrange multipliers
initial_lagrange_multipliers = [1.0]

# Bounds for Lagrange multipliers
bounds_lagrange_multipliers = [(0, None)] * len(initial_lagrange_multipliers)

# Initial guess
initial_guess = [0, 0]

# Bounds for variables
bounds = [(0, None), (0, None)]  # P1 and P2 are non-negative

# Solve using minimize with original objective and constraints
result = minimize(objective, initial_guess, bounds=bounds, constraints=[{'type': 'ineq', 'fun': constraint1},
                                                                       {'type': 'ineq', 'fun': constraint2},
                                                                       {'type': 'ineq', 'fun': constraint3}])

# Extract solution
p_solution = result.x
maximized_profit = -1 * result.fun  # Convert back to positive for interpretation

# Now that p_solution is defined, you can use it in the minimize function for Lagrange multipliers

result = minimize(lambda lagrange_multipliers: sum([val**2 for val in kkt_conditions(p_solution, lagrange_multipliers)]),
                  initial_lagrange_multipliers,
                  bounds=bounds_lagrange_multipliers)

# Extract Lagrange multipliers solution
lagrange_multipliers_solution = result.x

print("Optimal values:")
print("P1:", p_solution[0])
print("P2:", p_solution[1])
print("Maximized Revenue:", maximized_profit)
print("Lagrange Multipliers:", lagrange_multipliers_solution)

Optimal values:
P1: 383.8504532037101
P2: 2296.4968098122035
Maximized Revenue: 50154983.343558624
Lagrange Multipliers: [5.72813245e-08]


# Part b)

In [29]:
# Objective function
def objective_function(p1, p2):
    return p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2)

# Projected Gradient Descent with Gurobi
def projected_gradient_descent_with_gurobi(learning_rate, threshold):
    # Initialize Gurobi model
    model = gb.Model("projected_gradient_descent")
    
    # Decision variables
    p1 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p1")
    p2 = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name="p2")
    
    # Objective
    obj = p1 * (intercept_values[0,0] + slope[0,0] * p1) + p2 * (intercept_values[0,1] + slope[0,1] * p2)
    model.setObjective(obj, GRB.MAXIMIZE)
    
    # Optimization loop
    prev_obj = float('inf')
    i = 0
    
    while True:
        # Price Constraint  
        model.addConstr(p1 <= p2, f"Price Constraint {i}")

        model.addConstr(intercept_values[0,0] + slope[0,0]*p1 >= 0, "Demand Definition Product Line 1 Basic")
        model.addConstr(intercept_values[0,1] + slope[0,1]*p2 >= 0, "Demand Definition Product Line 1 Advance")
        
        # Optimize model
        model.optimize()
        
        # Get current solution
        current_p1 = p1.X
        current_p2 = p2.X
        
        # Compute objective function
        current_obj = objective_function(current_p1, current_p2)
        
        # Check convergence
        if (current_obj - prev_obj) < threshold:
            break
        
        prev_obj = current_obj
        
        # Compute gradient
        df_dx = intercept_values[0,0] + 2 * slope[0,0] * current_p1
        df_dy = intercept_values[0,1] + 2 * slope[0,1] * current_p2
        
        # Update parameters
        current_p1 -= learning_rate * df_dx
        current_p2 -= learning_rate * df_dy
        
        # Set new starting point
        p1.Start = current_p1
        p2.Start = current_p2
        
        # Increment iteration counter
        i += 1
    
    return current_p1, current_p2

# Hyperparameters
learning_rate = 0.001
threshold = 1e-6

# Run projected gradient descent with Gurobi
final_x, final_y = projected_gradient_descent_with_gurobi(learning_rate, threshold)
print(f"Final solution: x = {final_x}, y = {final_y}, Objective = {round(objective_function(final_x, final_y),10)}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x993e9687
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 4e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     

# Part c)

In [30]:
# Create the optimization model
part_c_model = gb.Model("Question 1c): TechEssentials Certain Product Line")

In [31]:
p = part_c_model.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name="Price")

In [32]:
#Objective Function
part_c_model.setObjective(gb.quicksum(p[i,n]*(intercept_values[i,n] + slope[i,n]*p[i,n]) for i in range(3) for n in range(3)), GRB.MAXIMIZE)

In [33]:
for i in range(3):
    for n in range(3):
        part_c_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) >= 0, "Demand Lower Bound")

In [34]:
# Price Constraint
for n in range(2):  
    part_c_model.addConstr(p[0, n] <= p[0, n + 1], f"Price Constraint {i}")
    part_c_model.addConstr(p[1, n] <= p[1, n + 1], f"Price Constraint {i}")
    part_c_model.addConstr(p[2, n] <= p[2, n + 1], f"Price Constraint {i}")

In [35]:
for i in range(3): 
    for n in range(3): 
        part_c_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) <= product_capacity[i,n], "Max Demand")

In [36]:
# Solve our model
part_c_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 24 rows, 9 columns and 30 nonzeros
Model fingerprint: 0x7c0c1e87
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl 

In [37]:
# Value of the objective function
print("Revenue: ", round(part_c_model.objVal,2))

Revenue:  718382097.67


In [38]:
# Print the decision variables
print(part_c_model.printAttr('X'))


    Variable            X 
-------------------------
  Price[0,0]      383.848 
  Price[0,1]       2296.5 
  Price[0,2]      2351.88 
  Price[1,0]       2050.3 
  Price[1,1]      4160.71 
  Price[1,2]      6813.66 
  Price[2,0]      5870.93 
  Price[2,1]      5870.93 
  Price[2,2]      8558.94 
None


# Part d)

In [39]:
# Create the optimization model
part_d_model = gb.Model("Question 1d): TechEssentials Certain Product Line")

In [40]:
p = part_d_model.addVars(3,3, lb=0, vtype=GRB.CONTINUOUS, name="Price")

In [41]:
#Objective Function
part_d_model.setObjective(gb.quicksum(p[i,n]*(intercept_values[i][n] + slope[i][n]*p[i,n]) for i in range(3) for n in range(3)), GRB.MAXIMIZE)

In [42]:
for i in range(3):
    for n in range(3):
        part_d_model.addConstr((intercept_values[i][n] + slope[i][n]*p[i,n]) >= 0, "Demand Lower Bound")

In [43]:
# Price Constraint
for n in range(2):  
    part_d_model.addConstr(p[0, n] <= p[0, n + 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[1, n] <= p[1, n + 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[2, n] <= p[2, n + 1], f"Price Constraint {i}")

for n in range(2):  
    part_d_model.addConstr(p[n, 0] <= p[n + 1, 0], f"Price Constraint {i}")
    part_d_model.addConstr(p[n, 1] <= p[n + 1, 1], f"Price Constraint {i}")
    part_d_model.addConstr(p[n, 2] <= p[n + 1, 2], f"Price Constraint {i}")

#part_d_model.addConstr(p[0, 1] <= p[1, 0], f"Price Constraint Additional 1")
#part_d_model.addConstr(p[0, 2] <= p[1, 0], f"Price Constraint Additional 2")
#part_d_model.addConstr(p[1, 1] <= p[2, 0], f"Price Constraint Additional 3")
#part_d_model.addConstr(p[1, 2] <= p[2, 0], f"Price Constraint Additional 4")

In [44]:
for i in range(3): 
    for n in range(3): 
        part_d_model.addConstr((intercept_values[i,n] + slope[i,n]*p[i,n]) <= product_capacity[i,n], "Max Demand")

In [45]:
# Solve our model
part_d_model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 30 rows, 9 columns and 42 nonzeros
Model fingerprint: 0x4ea948ad
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 18 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl

In [46]:
# Value of the objective function
print("Revenue: ", round(part_d_model.objVal,2))

Revenue:  718382097.67


In [47]:
# Print the decision variables
print(part_d_model.printAttr('X'))


    Variable            X 
-------------------------
  Price[0,0]      383.848 
  Price[0,1]       2296.5 
  Price[0,2]      2351.88 
  Price[1,0]       2050.3 
  Price[1,1]      4160.71 
  Price[1,2]      6813.66 
  Price[2,0]      5870.93 
  Price[2,1]      5870.93 
  Price[2,2]      8558.94 
None


In [48]:
# Extract values from the tupledict
values = [v for v in part_d_model.getAttr('x', p).values()]

# Reshape the values
price_values = np.array(values).reshape(3, -1)

price_values

array([[ 383.84827161, 2296.49891813, 2351.87766702],
       [2050.29879442, 4160.70785608, 6813.65650408],
       [5870.9328096 , 5870.9328096 , 8558.94236073]])

In [49]:
demand = [[intercept_values[i][n] + slope[i][n] * price_values[i][n] for n in range(3)] for i in range(3)]
demand

[[17617.272892756177, 18895.120416101, 17837.666608596355],
 [18520.690188945133, 18423.070192958312, 17913.51187327708],
 [25197.89694029585, 12505.211952096714, 19656.65851562613]]